In [9]:
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
import os
import json
import json_encoder as json_enc
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
import os
import json
import pickle
import ollama
import chromadb
import pickle
import pandas as pd
#rag_test_input_path='/home/matrix4284/MY_GEN_AI_PROJECTS/RAG/GraphRAG/graphrag-local-ollama/ragtest/input/'+file_name
import os
# importing shutil module
import shutil

from langchain_community.chat_models import ChatOllama

from langchain_community.embeddings import HuggingFaceEmbeddings

from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

#from sentence_transformers import SentenceTransformer
#embeddings = SentenceTransformer("all-MiniLM-L6-v2")

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

import json
import operator
from typing import Annotated, Sequence, TypedDict

from langchain import hub
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import Chroma
from langchain_core.messages import BaseMessage, FunctionMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import RunnablePassthrough
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
import json
import operator
from typing import Annotated, Sequence, TypedDict

from langchain import hub
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import Chroma
from langchain_core.messages import BaseMessage, FunctionMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import RunnablePassthrough
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import XMLOutputParser

In [10]:
import pandas as pd

single_df=pd.read_csv('Entity_Single_Occurence.csv')
single_df.head()

,Unnamed: 0,entity,description,category,node_occurence_flag,entity_embedding,entity_desc_embedding,category_embedding
0,6,UTS namespace,Shared hostname and network interfaces among c...,network,Single,"[-0.4460306465625763, -0.6342083215713501, 0.3...","[-0.3808414340019226, 0.1560606211423874, 0.33...","[-0.06634131073951721, 0.28101086616516113, -0..."
1,11,Port space,Shared port numbers among containers in a pod,network,Single,"[-0.1327187418937683, -0.023213177919387817, -...","[-0.3334151804447174, 0.2883642017841339, 0.01...","[-0.06634131073951721, 0.28101086616516113, -0..."
2,14,Flat inter-pod network,"single flat, shared network-address space for ...",network,Single,"[-0.2555774748325348, 0.6702328324317932, 0.31...","[-0.13991282880306244, 0.37147456407546997, 0....","[-0.06634131073951721, 0.28101086616516113, -0..."
3,18,NAT (Network Address Translation),gateway between pods that translates IP addresses,software,Single,"[-0.31485122442245483, -0.09275610744953156, 0...","[-0.6352800726890564, -0.09631533920764923, -0...","[0.16625313460826874, -0.33344849944114685, -0..."
4,19,Worker nodes,physical or virtual machines that run pods,hardware,Single,"[0.19375282526016235, -0.1520829200744629, -0....","[-0.1788252890110016, -0.37786591053009033, -0...","[-0.5788261890411377, -0.5416646003723145, -0...."


In [11]:
multi_df=pd.read_csv('Entity_Multi_Occurence.csv')
multi_df.head()

,Unnamed: 0,entity,description,description_raw_str,description_formatted,category,entity_embedding,entity_desc_embedding,category_embedding
0,0,$ kubectl get po,['A command used to display information about ...,A command used to display information about ru...,The `kubectl get pods` command is a powerful t...,container,"[0.0625637099146843, -0.09416717290878296, -0....","[0.25978609919548035, -0.16973280906677246, -0...","[-0.6506563425064087, -0.4570855498313904, 0.6..."
1,1,--all,['option to delete all resources of a certain ...,option to delete all resources of a certain ty...,The `delete` option allows for the removal of ...,process,"[-0.17758798599243164, 0.3256435692310333, 0.5...","[-0.07883873581886292, 0.23203814029693604, 0....","[-0.3130657970905304, -0.05992688983678818, -0..."
2,2,--all-namespaces,['option for listing resources across all name...,option for listing resources across all namesp...,Kubernetes provides an option to list resource...,Kubernetes,"[-0.18715298175811768, 0.43089669942855835, 0....","[0.10242711752653122, 0.24409592151641846, 0.2...","[0.33145245909690857, 0.14809204638004303, -0...."
3,3,--overwrite,['option to overwrite existing labels when mod...,option to overwrite existing labels when modif...,The `kubectl label` command offers a feature t...,process,"[0.15122950077056885, -0.12455412745475769, 0....","[-0.27481114864349365, -0.5290030241012573, -0...","[-0.3130657970905304, -0.05992688983678818, -0..."
4,4,/api,"['API path for Kubernetes API server', 'A path...",API path for Kubernetes API serverA path in th...,The Kubernetes API server exposes a specific p...,application,"[0.21717451512813568, 0.3137848377227783, -0.3...","[0.05292505398392677, 0.8759827613830566, -0.6...","[0.2571965456008911, -0.48218244314193726, -0...."


In [61]:
entity_df = pd.concat([single_df, multi_df], ignore_index=True)

entity_df['entity_embedding'][0].split(',')

['[-0.4460306465625763',
 ' -0.6342083215713501',
 ' 0.31488773226737976',
 ' -0.6608408689498901',
 ' -0.7309532761573792',
 ' -0.07249678671360016',
 ' -0.004192069172859192',
 ' -0.5568747520446777',
 ' 0.09961221367120743',
 ' 0.8677575588226318',
 ' 0.5612044930458069',
 ' 0.6025923490524292',
 ' 0.579443097114563',
 ' 0.6165585517883301',
 ' -0.6272537708282471',
 ' 0.06625895202159882',
 ' 0.23447684943675995',
 ' -0.3495834767818451',
 ' -0.9903642535209656',
 ' -0.5664210915565491',
 ' -0.018731333315372467',
 ' 0.3540385365486145',
 ' -1.0800925493240356',
 ' -0.8271225094795227',
 ' 0.05482247471809387',
 ' -0.1551765352487564',
 ' -0.13533847033977509',
 ' 0.17612779140472412',
 ' 0.838219404220581',
 ' 1.1204819679260254',
 ' -0.3695462644100189',
 ' -0.4967446029186249',
 ' 0.023111041635274887',
 ' -0.42766299843788147',
 ' -0.5266684889793396',
 ' -0.48819467425346375',
 ' 0.259791761636734',
 ' 0.2330821454524994',
 ' 0.0549662820994854',
 ' -0.651677668094635',
 ' 0.6

In [16]:
client = chromadb.Client()

entity_collection = client.create_collection(name="entity_collection")

In [37]:
def embedding_put(idx,name,embedding):
    entity_collection.add(
    ids=[str(idx)],
    embeddings=list(embedding.split(',')).replace("''","")
    documents=[name]
  )
    return True

In [38]:
entity_df=entity_df.reset_index(drop=True)

In [39]:
entity_df.columns

Index(['index', 'Unnamed: 0', 'entity', 'description', 'category',
       'node_occurence_flag', 'entity_embedding', 'entity_desc_embedding',
       'category_embedding', 'description_raw_str', 'description_formatted'],
      dtype='object')

In [40]:
entity_df_final=entity_df[['index', 'entity', 'description', 'category',
       'node_occurence_flag', 'entity_embedding', 'entity_desc_embedding',
       'category_embedding', 'description_raw_str', 'description_formatted']]

In [41]:
entity_df_final['index']

0          0
1          1
2          2
3          3
4          4
        ... 
2590    2590
2591    2591
2592    2592
2593    2593
2594    2594
Name: index, Length: 2595, dtype: int64

In [42]:
entity_df_final[['index','entity','entity_embedding']].apply(lambda x : embedding_put(x.index,x.entity,x.entity_embedding),axis=1)

ValueError: Expected embeddings to be a list, got [-0.4460306465625763, -0.6342083215713501, 0.31488773226737976, -0.6608408689498901, -0.7309532761573792, -0.07249678671360016, -0.004192069172859192, -0.5568747520446777, 0.09961221367120743, 0.8677575588226318, 0.5612044930458069, 0.6025923490524292, 0.579443097114563, 0.6165585517883301, -0.6272537708282471, 0.06625895202159882, 0.23447684943675995, -0.3495834767818451, -0.9903642535209656, -0.5664210915565491, -0.018731333315372467, 0.3540385365486145, -1.0800925493240356, -0.8271225094795227, 0.05482247471809387, -0.1551765352487564, -0.13533847033977509, 0.17612779140472412, 0.838219404220581, 1.1204819679260254, -0.3695462644100189, -0.4967446029186249, 0.023111041635274887, -0.42766299843788147, -0.5266684889793396, -0.48819467425346375, 0.259791761636734, 0.2330821454524994, 0.0549662820994854, -0.651677668094635, 0.6314529180526733, 0.05369739234447479, -0.3050019443035126, -0.2609148323535919, -0.5331178903579712, 0.632784247